In [4]:
import json
import random

import nltk
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os

#nltk.download('punkt_tab')

In [ ]:
"""# load the dataset
with open('data/healthcare.json','r',encoding='utf-8') as f:
    data = json.load(f)

intents = {'intents':[]}
#
for convo in data:
    tag = convo.get("agent_selected_tool", "general").replace(" ","_").lower()


    pattern = convo.get("user_1", "")
    response = convo.get("agent_initial_response", "")


    if pattern and response:
        intents['intents'].append({
            "tag":tag,
            "patterns":[pattern],
            "responses":[response]
        })

# Save the new dataset
with open("data/healthcare_intents.json", "w", encoding="utf-8") as f:
    json.dump(intents, f, indent=4)


print('Dataset intentions create with sucess!')"""
    








Dataset intentions create with sucess!


In [3]:
for resource in ['wordnet', 'omw-1.4', 'punkt']:
    try:
        nltk.data.find(f'corpora/{resource}')
    except LookupError:
        nltk.download(resource)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Calixte\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Calixte\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Calixte\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [2]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

with open("data/healthcare_intents.json", "r") as f:
    data = json.load(f)

# Vocabulaire
vocab = []
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        vocab.extend([lemmatizer.lemmatize(word.lower()) for word in tokens])

vocab = sorted(set(vocab))
input_size = len(vocab)

# Intentions
intents = [intent["tag"] for intent in data["intents"]]
output_size = len(set(intents))

print(f"Input size (vocabulaire): {input_size}")
print(f"Output size (intentions): {output_size}")

Input size (vocabulaire): 990
Output size (intentions): 137


In [ ]:
class ChatboModel(nn.Module):
    
    def __init__(self, input_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128,output_size)
        
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

        


In [ ]:
class ChabotAssistant:
    def __init__(self,intents_path, function_mappings = None):
        self.model = None
        self.intents_path = intents_path

        self.documents = []
        self.vocabulary = []
        self.intents = []
        self.intents_responses = []

        self.function_mappings = function_mappings

        self.X = None
        self.y = None


    @staticmethod
    def tokenize_and_lemmatize(text):
        lemmatizer = nltk.WordNetLemmatizer()

        words = nltk.word_tokenize(text)
        words = [lemmatizer.lemmatize(words.lower()) for word in words]

        return words
    
    def bag_of_words(self, words):
        return[1 if word in words else 0 for word in self.vocabulary]
    
    def parse_intents(self):
        lemmatizer = nltk.WordNetLemmatizer()

        if os.path.exits(self.intents_path):
            with open(self.intents_path,'r') as f:
                intents_data = json.load(f)

            for intent in intents_data['intents']:
                if intent['tag'] not in self.intents:
                    self.intents.append(intent['tag'])
                    self.intents_responses[intent['tag']] = intent['responses']
                
                for pattern in intent['patterns']:
                    pattern_words = self.tokenize_and_lemmatize(pattern)
                    self.vocabulary.extend(pattern_words)
                    self.documents.append((pattern_words, intent['tag']))
                
                self.vocabulary = sorted(set(self.vocabulary))

    
    def prepare_data(self):
        bags = []
        indices = []

        for document in self.documents:
            words = document[0]
            bag = self.bag_of_words(words)

            intent_index = self.index(document[1])

            bags.append(bag)
            indices.append(intent_index(document[1]))

            bags.append(bags)
            indices.append(intent_index)

        self.X = np.array(bags)
        self.y = np.array(indices)


    def train_model(self, batch_size, lr, epochs):
        X_tensor = torch.tensor(self.X,dtype = torch.float32)
        y_tensor = torch.tensor(self.y,dtype = torch.long)
        
        dataset = TensorDataset(X_tensor,y_tensor)
        loader  = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        self.model = ChatboModel(self.X.shape[1],len(self.intents))

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr = lr)

        for epoch in range(epochs):
            running_loss = 0.0

            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                running_loss += loss

            print(f"Epoch {epoch+1}: Loss: {running_loss/len(loader):.4f}")

    
    def save_model(self,model_path, dimensions_path):
        torch.save(self.model.state_dict(), model_path)

        with open(dimensions_path,"r") as f:
            json.dump({'input_size': self.X.shape[1],'output_size':len(self.intents)},f)

    def load_model(self,model_path, dimensions_path):
        with open(dimensions_path,"r") as f:
            dimensions = json.load(f)

        self.model = ChatboModel(dimensions['input_size'], dimensions['output_size'])
        self.model.load_state_dict(torch.load(model_path, weights_only=True))

    def process_message(self, input_message):
        words = self.tokenize_and_lemmatize(input_message)
        bag = self.bag_of_words(words)

        bag_tensor = torch.tensor([bag], dtype=torch.float32)

        self.model.eval()
        with torch.no_grad():
            predictions = self.model(bag_tensor)

        predicted_class_index = torch.argmax(predictions, dim=1).item()
        predicted_intent = self.intents[predicted_class_index]

        if self.function_mappings:
            if predicted_intent in self.function_mappings:
                self.function_mappings[predicted_intent]()
        
        if self.intents_responses[predicted_intent]:
            return random.choice(self.intents_responses[predicted_intent])
        else:
            return None






    

In [ ]:
if __name__== '__main__':
    assistant = ChabotAssistant()
    assistant.parse_intents()
    assistant.prepare_data()
    assistant.train_model(batch_size=32, lr =0.001, epochs=150)    

    assistant.save_model('chat_model.pth','dimensions.json')


    #assistant = assistant.load_model('chat_model.pth','dimensions.json')

    while True:
        message = input('Enter your message:')

        if message == '/quit':
            break

        print(assistant.process_message(message))